In [ ]:
from DocumentProcessing.pdf_processing import pdf_processor
from DocumentProcessing.pdf_processing import paper_processor

In [ ]:
pdf_paper = "/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/1810.04805.pdf"

In [ ]:
full_text, title,publish_display_data, authors, abstract, references = paper_processor.extract_text_from_pdf(pdf_paper)

In [ ]:
full_text = pdf_processor.extract_text(pdf_paper)

In [ ]:
full_text[1]

In [ ]:
pdf_processor.extract_toc_from_pdf(pdf_paper)

In [ ]:
full_text_2[3]

In [ ]:
pdf_processor.extract_pdf_metadata(pdf_paper)

In [ ]:
# pdf_processor.py

import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path, max_non_text_pages=10):
    with fitz.open(pdf_path) as doc:
        text = ''
        non_text_page_count = 0

        for page in doc:
            page_text = page.get_text()
            if not page_text:
                non_text_page_count += 1
                if non_text_page_count >= max_non_text_pages:
                    raise ValueError("PDF contains too many non-text pages. Please use your own OCR tool first.")
                continue
            else:
                non_text_page_count = 0  # Reset counter if a text page is found
            text += page_text

        return text

def extract_toc_from_pdf(pdf_path, max_non_text_pages=10):
    with fitz.open(pdf_path) as doc:
        toc = doc.get_toc(simple=False)
        non_text_page_count = 0

        # Check for non-text pages in the document
        for page in doc:
            if not page.get_text():
                non_text_page_count += 1
                if non_text_page_count >= max_non_text_pages:
                    raise ValueError("PDF contains too many non-text pages. Please use your own OCR tool first.")
                continue
            else:
                non_text_page_count = 0  # Reset counter if a text page is found
                break  # Break the loop after finding the first text page

        return toc


def extract_notes_from_pdf(pdf_path):
    notes = []
    with fitz.open(pdf_path) as doc:
        for page in doc:
            annotations = page.annots()
            if annotations:
                for annot in annotations:
                    annot_text = annot.info['content']
                    if annot_text:
                        notes.append(annot_text)
    return notes


def extract_pdf_metadata(pdf_path):
    with fitz.open(pdf_path) as doc:
        metadata = doc.metadata
    return metadata

# Example usage
if __name__ == "__main__":
    pass


def extract_pdf_metadata(file_path):
    """
    Extracts metadata from a PDF file.
    Args:
    file_path (str): Path to the PDF file.
    Returns:
    dict: Metadata of the PDF.
    """
    metadata = {}
    with pdfplumber.open(file_path) as pdf:
        metadata = pdf.metadata
    return metadata

In [ ]:
from DocumentManagement.Elastic import IndexSettingsGenerator,search_engine

In [ ]:
from elasticsearch import Elasticsearch, helpers

In [ ]:
es = Elasticsearch(
    ['https://localhost:9200'],
    http_auth=('elastic', 'dR8dVIqQ5=i3pPSH00zC'),  # Replace with your credentials
    verify_certs=True,
    ca_certs='/root/http_ca.crt'  # Path to your CA certificate
)


In [ ]:
from tika import parser

parsed = parser.from_file(pdf_paper)
print(parsed["metadata"])  # To print the metadata of the document.
print(parsed["content"])   # To print the text content of the document.


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)

    # Extract the content
    content = parsed['content']
    if not content:
        raise ValueError("No content found in the PDF file.")

    # Split the content into pages
    pages = content.split('\f')  # Splitting by form feed character which typically represents page breaks
    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, 1):
        # Clean up whitespace
        page_text = ' '.join(page_content.strip().split())

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("PDF seems to contain only images. Please use OCR first.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text

# Usage
# file_path = 'path_to_your_pdf_file.pdf'
# pdf_text = extract_text_tika(file_path)


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)

    # Extract the content
    content = parsed['content']
    if not content:
        raise ValueError("No content found in the PDF file.")

    # Split the content into pages
    pages = content.split('\f')  # Splitting by form feed character which typically represents page breaks
    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, 1):
        # Clean up whitespace
        page_text = ' '.join(page_content.strip().split())

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("PDF seems to contain only images. Please use OCR first.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text

# Usage
# file_path = 'path_to_your_pdf_file.pdf'
# pdf_text = extract_text_tika(file_path)


In [ ]:
from tika import parser
import re

def extract_text_tika(file_path):
    """
    Extracts text from a PDF file using Apache Tika, page by page, and checks for consecutive pages without text.
    Args:
        file_path (str): Path to the PDF file.
    Returns:
        dict: A dictionary where keys are page numbers and values are extracted text from each page.
    Raises:
        ValueError: If 10 consecutive pages are found without text.
    """
    # Parse the PDF file
    parsed = parser.from_file(file_path)
    content = parsed['content']

    # Split the content by pages - Tika's plain text output may not provide clear page delimiters
    # Adjust the splitting logic based on the actual output format
    pages = content.split('some_page_delimiter')  # Replace 'some_page_delimiter' with the actual delimiter used by Tika

    text = {}
    consecutive_pages_without_text = 0

    for i, page_content in enumerate(pages, start=1):
        page_text = page_content.strip()

        if not page_text:
            consecutive_pages_without_text += 1
            if consecutive_pages_without_text == 10:
                raise ValueError("10 consecutive pages without text found. PDF may contain only images. Please use OCR.")
        else:
            consecutive_pages_without_text = 0

        text[i] = page_text

    return text


In [ ]:
pdf_text = extract_text_tika(pdf_paper)

In [ ]:
pdf_paper

In [ ]:
pdf_text

In [ ]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO

def add_invisible_markers(pdf_path, marker):
    reader = PdfReader(pdf_path)
    writer = PdfWriter()

    for i in range(len(reader.pages)):
        page = reader.pages[i]
        packet = BytesIO()
        can = canvas.Canvas(packet, pagesize=letter)
        can.setFontSize(1)  # Setting font size to 1 or 0 to make it nearly invisible
        can.drawString(0, 0, marker)  # Position at bottom-left; can adjust as needed
        can.save()

        packet.seek(0)
        new_pdf = PdfReader(packet)
        page.merge_page(new_pdf.pages[0])
        writer.add_page(page)

    with open("/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/modified_pdf.pdf", "wb") as f:
        writer.write(f)

add_invisible_markers(pdf_paper ,"__PageBreak__12345__")



In [ ]:
from tika import parser

def extract_pages_with_tika(file_path, marker):
    parsed = parser.from_file(file_path)
    content = parsed['content']
    pages = content.split(marker)  # Splitting text using the marker

    text_by_page = {i+1: page.strip() for i, page in enumerate(pages)}
    return text_by_page

text_by_page = extract_pages_with_tika("/root/gpt_projects/ABoringKnowledgeManagementSystem/DocumentProcessing/tests/modified_pdf.pdf", "__PageBreak__12345__")


In [ ]:
text_by_page

In [ ]:
full_text = pdf_processor.extract_text(pdf_paper)

In [ ]:
text_by_page[1], full_text[1]

In [ ]:
from DocumentIndexing.Elastic import IndexSettingsGenerator
from DocumentIndexing.Elastic import search_engine
from elasticsearch.helpers import scan

In [ ]:
from config import *

In [ ]:
from DocumentIndexing.Embedding import embedding_local

In [ ]:
indexsetting = IndexSettingsGenerator.GeneralIndexSettings(EMBEDDING_DINENSION)
mapping = indexsetting.get_base_mapping()

In [ ]:
mapping

In [ ]:
eleasticengine = search_engine.SearchEngine()

In [ ]:
for index_name in DOCUMENT_TYPE:
    eleasticengine.create_index(index_name,mapping)

In [ ]:
index_name = "research_paper"

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

# Connect to Elasticsearch
es = Elasticsearch(hosts=ES_HOST, http_auth=ES_HTTP_AUTH, verify_certs=True, ca_certs=ES_CA_CERTS_PATH)
# Index name
index_name = "research_paper"

# Initialize the scan
results = scan(es, index=index_name, query={"query": {"match_all": {}}})

# Iterate over the results
for doc in results:
    print(doc)


In [17]:
res = eleasticengine.search_for_terms("research_paper",'ﬁnetuning')

In [ ]:
res['hits']

In [12]:
vector = embedding_local.embeddings_multilingual("ﬁnetuning")

In [18]:
res2 = eleasticengine.vector_search("research_paper",vector)

In [19]:
res2['hits']

{'total': {'value': 16, 'relation': 'eq'},
 'max_score': 1.0799596,
 'hits': [{'_index': 'research_paper',
   '_id': '2610a7e7-359f-4e79-85d8-d3e3dff51dab_6_0',
   '_score': 1.0799596,
   '_source': {'document_id_universal': '2610a7e7-359f-4e79-85d8-d3e3dff51dab',
    'upload_date': 1702506147687,
    'language': 'en',
    'metadata': {'Author': '',
     'ModDate': '28-05-2019',
     'Keywords': '',
     'Title': '',
     'Subject': ''},
    'document_id_elastic': '2610a7e7-359f-4e79-85d8-d3e3dff51dab_6_0',
    'original_page_number': 6,
    'text_piece': 'System\nPreOpenAI SOTA\nBiLSTMELMoAttn\nOpenAI GPT\nBERTBASE\nBERTLARGE\nMNLImm\n392k\n8080\n7676\n8281\n8483\n8685\nQQP\n363k\n66\n64\n70\n71\n72\nQNLI\n108k\n82\n79\n87\n90\n92\nSST\n67k\n93\n90\n91\n93\n94\nCoLA\n5k\n35\n36\n45\n52\n60\nSTS MRPC\n7k\n81\n73\n80\n85\n86\n5k\n86\n84\n82\n88\n89\nRTE\n5k\n61\n56\n56\n66\n70\nAverage\n74\n71\n75\n79\n82\nTable  GLUE Test results scored by the evaluation server httpsgluebenchmarkcomlea